In [180]:
from copy import deepcopy

# 1 - Sequence alignment

Genetic material such as DNA and proteins comes in sequences made up of different blocks. The study of these sequences and their correlation is invaluable to the field of biology. Since genetic material is likely to mutate over time, it is not easy to be certain of these correlations in most cases. In order to know if two or more sequences have a common origin, we must examine their similarity in a formal way, and be able to quantify it. Since the modifications of the sequences can introduce new blocks as well as delete some, the first step to comparing sequences is aligning them by making the similar regions match. Even then, high levels of similarity do not necessarily imply a common ancestor exists (homology). Our goal in this section is to create a tool that allows us to perform such alignments and give a numeric value of the chance of homology.


## ADTs

We will first look at an implementation of Abstract Data Types (ADTs) that will allow us to represent Sequences. 

### AminoAcid

The choice of genetic material is proteins. In the case of proteins, the blocks that make up the sequences are called amino acids. Therefore, they are the first thing we must be able to represent. The following tuple lists all amino acids and miscellaneous options.

In [181]:
#All the names of amino acids, plus gaps and some (partially) undetermined blocks
nameGroups = (
		("none/gap", "gap", "-"),
		("alanine", "ala", "A"),
		("cysteine", "cys", "C"),
		("aspartic acid", "asp", "D"),
		("glutamic acid", "glu", "E"),
		("phenylalanine", "phe", "F"),
		("glycine", "gly", "G"),
		("histidine", "his", "H"),
		("isoleucine", "ile", "I"),
		("lysine", "lys", "K"),
		("leucine", "leu", "L"),
		("methionine", "met", "M"),
		("asparagine", "asn", "N"),
		("proline", "pro", "P"),
		("glutamine", "gln", "Q"),
		("arginine", "arg", "R"),
		("serine", "ser", "S"),
		("threonine", "thr", "T"),
		("valine", "val", "V"),
		("tryptophan", "trp", "W"),
		("tyrosine", "tyr", "Y"),
		("asparagine/aspartic acid", "asx", "B"),
		("glutamine/glutamic acid", "glx", "Z"),
		("leucine/isoleucine", "xle", "J"),
		("selenocysteine", "sec", "U"),
		("pyrrolysine", "pyl", "O"),
		("undetermined", "und", "X")
	)

The following class allows us to create AminoAcid objects that represent any of the amino acids from the list. We can then compare, print, hash, test them with any of the defined methods. Do note that we can create them by copying other AminoAcids or by providing their name in any form (long for complete, medium for 3 chars or short for 1 char).

In [182]:
class AminoAcid:
	"""
	Represents one of the amino acids that can be found in genetic sequences.
	Can be one of the twenty-two amino acids, four undetermined combinations of possible amino acids, and gaps.
	"""
	#Tuple of tuples listing amino acid names
	_nameGroups = deepcopy(nameGroups)
	
	#Dictionary mapping name to id
	_nameDict = {nameGroups[id][i]:id for i in range(3) for id in range(len(nameGroups))}
	
	_nameModes = {"long":0, "medium":1, "short":2} #choices for name lenght
	_defaultNameMode = "short" #short name by default
	
	
	def __init__(self, aminoAcid):
		"""
		Creates an AminoAcid object representing one of the possible Amino Acids.
		aminoAcid can be the name of an amino acid, or an AminoAcid object (in which case a copy is created).
		"""
		self._id = None	#id of the amino acid within the name group
		
		if isinstance(aminoAcid, str):
			self._id = self.__getIdByName(aminoAcid) #id found from aminoAcid name
		elif isinstance(aminoAcid, AminoAcid):
			self._id = aminoAcid._id #copy of id
		else:			
			raise TypeError("aminoAcid must be a string or an AminoAcid object")
		
	
	@staticmethod
	def __getIdByName(name):
		try:
			return AminoAcid._nameDict[name] #get index of name mode
		except:
			raise ValueError("Could not find amino acid name {}".format(name))
			
	@staticmethod
	def getAllNames(nameMode=_defaultNameMode):
		try:
			nameIndex = AminoAcid._nameModes[nameMode] #get index of name mode
		except:
			raise TypeError("nameMode must be 'short', 'medium' or 'long'")
			
		for aa in AminoAcid._nameGroups[1:]: #we exclude the gap (first item)
			yield aa[nameIndex]
	
	#Representation
	def __repr__(self):
		nameIndex = AminoAcid._nameModes[self._defaultNameMode]
		return self._nameGroups[self._id][nameIndex] #default name mode
		
	def __str__(self):
		return self.getName() #default name mode
	
	def getName(self, nameMode=_defaultNameMode):
		try:
			nameIndex = AminoAcid._nameModes[nameMode] #get index of name mode
		except:
			raise TypeError("nameMode must be 'short', 'medium' or 'long'")
		
		return self._nameGroups[self._id][nameIndex]
	
	#Comparison		
	def __eq__(self, other):
		return self._id == other._id
	
	def __ne__(self, other):
		return self._id != other._id
		
	def __gt__(self, other):
		return self._id > other._id
		
	def __ge__(self, other):
		return self._id >= other._id
		
	def __lt__(self, other):
		return self._id < other._id
	
	def __le__(self, other):
		return self._id <= other._id
		
	def isGap(self):
		return self._id == 0
	
	
	#Hashing
	def __hash__(self):
		return hash(self._id)

Let's test it a little bit.

In [183]:
a1 = AminoAcid("A")
print(a1)
a2 = AminoAcid(a1)
print(a1 == a2)
a3 = AminoAcid("K")
print(a3.getName("long"))

A
True
lysine


### Sequence

That's good and all, but we don't want to align single amino acids do we ? What we want to align is sequences. We can view a Sequence as an ordered list of AminoAcid objects. It should be iterable : we'll want to go through, access, and count its items. We'll also want to change them, insert new ones, delete some, and do all that transparently, as we would with a list. Finally it would be useful to check whether a sequence contains some other sub-sequence or single amino acid. The following is a class that does just that.

In [184]:
class Sequence:
	"""
	Represents a sequence of amino acids.
	"""
	
	def __init__(self, aminoAcids=None, description=""):
		"""
		Creates a Sequence object that represents the amino acid sequence contained in aminoAcids.
		aminoAcids can be one of the following :
		- None, meaning the Sequence is empty (default)
		- an AminoAcid object
		- a string of X AminoAcid short (uppercase) names or 1 AminoAcid name
		- a list containing AminoAcid objects and/or strings of individual AminoAcid names
		"""
		
		self._nameMode = "short" #the way in which AA names are displayed
		self._separator = "" #how to separate AA names when displayed
		self._description = description #description of the sequence

		#Format aminoAcids into a list of AminoAcid objects.
		self._aaList = self.__formatAAList(aminoAcids) #List of amino acids
	
	
	@staticmethod
	def __formatAAList(aminoAcids):
		"""Formats 'aminoAcids' into a list of AminoAcid objects."""
		
		#Sequence object's aaList is deep copied
		if isinstance(aminoAcids, Sequence):
			return deepcopy(aminoAcids._aaList)
		
		#None becomes an empty Sequence
		elif aminoAcids is None:
			return []
		
		#A string is converted to a list
		elif isinstance(aminoAcids, str):
			if aminoAcids.isupper(): #Multiple Amino Acids in short name mode
				return [AminoAcid(aa) for aa in aminoAcids]
			else: #A single Amino Acid with any name mode
				return [AminoAcid(aminoAcids)]
		
		#An AminoAcid is copied (from name) and put within a list
		elif isinstance(aminoAcids, AminoAcid):
			return [AminoAcid(aminoAcids)] #Copy constructor
		
		#A list is copied with all of its items converted to AminoAcid objects
		elif isinstance(aminoAcids, list):
			return [AminoAcid(aa) for aa in aminoAcids]
		
		#No more supported types
		else:
			raise TypeError("aminoAcids must be a Sequence, list, AminoAcid object, string or None")
	
	#Size and comparison
	def __len__(self):
		return len(self._aaList)
		
	def __gt__(self, other):
		return len(self) > len(other)
		
	def __lt__(self, other):
		return len(self) < len(other)
		
	def __ge__(self, other):
		return len(self) >= len(other)
		
	def __le__(self, other):
		return len(self) <= len(other)
	
	def __eq__(self, other):
		return len(self) == len(other) and all(a == b for a, b in zip(self, other))
	
	def __ne__(self, other):
		return not self == other
	
	#Iteration
	def __iter__(self):
		return iter(self._aaList)
	
	#Representation
	def __repr__(self):
		"""Representation"""
		return str(self)
		
	def __str__(self):
		"""String conversion"""
		return self._separator.join([aa.getName(self._nameMode) for aa in self])
		
	def getDescription(self):
		"""Returns the description of the sequence."""
		return self._description
	
	def setNameMode(self, newMode):
		"""Changes the name display mode to 'newMode'."""
		if newMode in ("long", "medium", "short"):
			self._nameMode = newMode
		else:
			raise ValueError("newMode must be 'long', 'medium' or 'short'")
	
	def setSeparator(self, newSep):
		"""Changes the string that separates each displayed AminoAcid."""
		self._separator = newSep
	
	#Item and slice manipulation	
	def __getitem__(self, key):
		"""Return a Sequence object containing copies of the items from the slice"""
		if isinstance(key, slice):
			return Sequence([self._aaList[index] for index in range(*key.indices(len(self)))])
		else:
			try:
				return self._aaList[key] #Return the item of index 'key'
			except:
				raise ValueError("key does not represent an index or slice")
		
	def __setitem__(self, key, value):		
		"""Set value for a slice of the sequence"""
		if isinstance(key, slice):
			for index in range(*key.indices(len(self))): #range(start, stop, step)
				self._aaList[index] = AminoAcid(value) #create copies
		else:
			try:
				self._aaList[key] = AminoAcid(value) #Set value for one item
			except:
				raise ValueError("key does not represent an index or slice")
		
	def __delitem__(self, key):
		"""Delete a slice of the sequence"""
		if isinstance(key, slice):
			start, stop, step = key.indices(len(self))
			del self._aaList[start:stop:step]
		else:
			try:
				del self._aaList[key] #Delete one item
			except:
				raise ValueError("key does not represent an index or slice")
	
	
	#Sequence modification
	def insert(self, subSequence, index=0):
		"""
		Inserts subSequence into the Sequence at index 'index' (default is 0).
		subSequence must be compatible with an AminoAcid list, as specified by __formatAAList.
		"""
		#We need a formatted sequence
		for aa in self.__formatAAList(subSequence):
			self._aaList.insert(index, aa)
			index += 1
			
	def extend(self, subSequence):
		"""
		Same as calling insert at the end of the Sequence.
		"""
		self.insert(subSequence, len(self))
		
			
	def remove(self, subSequence, startIndex=0):
		"""
		Removes the first occurrence of 'subSequence' in self, starting at index. Returns
		- start index of the sub-sequence within self (if it exists)
		- False if subSequence is not a sub-sequence of self
		"""
		lookupResult = self.hasSubSequence(subSequence, startIndex)
		if isinstance(lookupResult, int):
			del self[startIndex:startIndex+len(subSequence)]
		
		return lookupResult
	
			
	def delete(self, start=0, stop=None, step=None):
		"""
		Deletes Amino Acids between indexes start (included) and stop (excluded).
		If start is not specified, deletion will begin at index 0.
		If stop is not specified, deletion will stop after one item.
		"""
		if stop is None:
			stop = start + 1
		if step is None:
			step = 1
		del self[start:stop:step]
	
	
	#Lookup
	def __contains__(self, item):
		"""
		Returns True if item is contained in Sequence, False if not.
		"""
		return item in self._aaList
	
	def hasSubSequence(self, subSequence, startIndex=0):
		"""
		Checks if 'subSequence' is a sub-sequence of self, starting at startIndex. Returns
		- start index of the sub-sequence within self, if it exists
		- False if sequence is not a sub-sequence of self
		"""
		subLen = len(subSequence)
		endIndex = len(self)-subLen+1
		if endIndex < startIndex or startIndex < 0:
			raise ValueError("subSequence does not fit in sequence after startIndex")
			
		for i in range(startIndex, endIndex):
			if all(self[i+j] == subSequence[j] for j in range(subLen)):
				return i
		
		return False

Since we may not want to type every sequence we use by hand, we can also read them from files. The format chosen here is .fasta, but this can be adapted for any other format.

In [185]:
def loadFasta(path):
	"""
	Loads the fasta file located in 'path' and yields the Sequences it contains.
	"""
	with open(path, 'r') as fastaFile:
		newSequence = None
		for line in fastaFile:
			line_s = line.strip()
			if line_s != "" and line_s[0] == ">":
				if newSequence is not None:
					yield newSequence
				newSequence = Sequence(None, line_s[1:])
			else:
				newSequence.extend(line_s)
		if len(newSequence)>0:
			yield newSequence

Let's test a few of the capabilities of this class :

In [186]:
s = Sequence("ABX")
print(s)
s.extend("cysteine")
print(s)
print(AminoAcid("alanine") in s)
sCopy = Sequence(s) #This is a deep copy
sAlias = s #This is the same object
del s[1:3]
sAlias[0] = "K"
s.setSeparator("-")
s.setNameMode("long")
print(s, sAlias, sCopy, sep=", ")

sequences = [seq for seq in loadFasta("PDZ-sequences.fasta")]
print(sequences[0])
print(sequences[0].hasSubSequence(Sequence("IGDDPSIFITKIIPGGA"))) #Returns the index of the sub-sequence, if found

ABX
ABXC
True
lysine-cysteine, lysine-cysteine, ABXC
EITLERGNSGLGFSIAGGTDNPHIGDDPSIFITKIIPGGAAAQDGRLRVNDSILFVNEVDVREVTHSAAVEALKEAGSIVRLYVMRR
23


### Score

Now that the easy part is done, we need to remember why we're here (it's sequence alignment, just in case). We can align two sequences any number of ways, but we're only interested in alignments that could represent two descendants of the same original sequence. Therefore, we need to assign a **score** value to each alignment, that will help us see how significative it is.

Because of the way mutations happen, all modifications should not be treated equally. Some amino acids are more related between them than others, therefore making them more likely to mutate into each other. Some changes are more or less likely to happen because of the very structure of the protein. Also, when an amino acid is inserted into or deleted from a sequence (thus creating a **gap** in the alignment), it usually is not the only one. This tells us that, not only should the score of the alignment depend on each pair that's aligned (meaning, on the same location), but it should also depend on how many gaps there are and how big they are.

The `Score` class allows us to give a numerical value to each possible pair of amino acids. These values can be set manually one by one, just as they can be loaded from files. The chosen format here was `.iij`.

In [187]:
class Score:
	"""
	Represents a scoring matrix, used to determine the score between two Amino Acids
	"""
	
	def __init__(self, path="", description="", ignore=None):
		"""
		Creates a Score object.
		If 'path' is provided, loads the Score values from an iij file.
		Otherwise, creates a Score for all possible AminoAcids with values 0.
		"""
		self._description = description
		self._ignore = Sequence(ignore)
		self._matrix = []
		self._aaOrder = {}
		self._aaSequence = Sequence()
		
		#If path is provided, load directly from iij file
		if path != "":
			with open(path, 'r') as file:
				foundAAOrder = False #Have we found the line with the amino acid values and order yet?
				for line in file:
					if line[0] != "#": #Comments
						
						if not foundAAOrder: #Read aa values and order
							for aa in line.split():
								self._aaSequence.extend(aa)
							self._aaOrder = {aa: index for aa, index in zip(self._aaSequence, range(len(self._aaSequence)))}
							foundAAOrder = True
						else: #Read matrix values
							self._matrix.append([int(v) for v in line.split()])
		
		#Otherwise initialize matrix with 0
		else:
			lineSize = 1
			for aa in AminoAcid.getAllNames():
				if AminoAcid(aa) not in self._ignore:
					self._aaSequence.extend(aa)
					self._aaOrder[self._aaSequence[-1]] = lineSize-1
					self._matrix.append([0 for i in range(lineSize)])
					lineSize += 1
				
	#Representation
	def __repr__(self):
		"""
		Representation.
		"""
		sepSize = 4
		result = ["---------- " + self._description + " ----------"]
		for values, aa in zip(self._matrix, self._aaSequence):
			tempstr = '{a!s:<{w}}'.format(a=aa, w=sepSize)
			for value in values:
				tempstr += '{v:<{w}}'.format(v=value, w=sepSize)
			result.append(tempstr)
		tempstr = " "*sepSize
		for aa in self._aaSequence :
			tempstr += '{a!s:<{w}}'.format(a=aa, w=sepSize)
		result.append("")
		result.append(tempstr)
		return "\n".join(result)
	
	#Scoring
	def setScore(self, aa1, aa2, score):
		"""
		Set the score assigned to AminoAcids 'aa1', 'aa2'.
		"""
		id1 = self._aaOrder[aa1]
		id2 = self._aaOrder[aa2]
		if id1 > id2:
			self._matrix[id1][id2] = score
		else:
			self._matrix[id2][id1] = score
		
	def getScore(self, aa1, aa2):
		"""
		Get the score assigned to AminoAcids 'aa1', 'aa2'.
		"""
		id1 = self._aaOrder[aa1]
		id2 = self._aaOrder[aa2]
		if id1 > id2:
			return self._matrix[id1][id2]
		else:
			return self._matrix[id2][id1]

Here's an example on how to load a `Score` object from a `.iij` file, and access the scores for certains pairs :

In [192]:
scoring = Score("blosum62.iij", "BLOSUM 62")
print(scoring)
a1, a2 = Sequence("HN") #That's call unpacking, pretty neat huh ?
print(a1, a2, " : ", scoring.getScore(a1, a2))

---------- BLOSUM 62 ----------
A   4   
R   -1  5   
N   -2  0   6   
D   -2  -2  1   6   
C   0   -3  -3  -3  9   
Q   -1  1   0   0   -3  5   
E   -1  0   0   2   -4  2   5   
G   0   -2  0   -1  -3  -2  -2  6   
H   -2  0   1   -1  -3  0   0   -2  8   
I   -1  -3  -3  -3  -1  -3  -3  -4  -3  4   
L   -1  -2  -3  -4  -1  -2  -3  -4  -3  2   4   
K   -1  2   0   -1  -3  1   1   -2  -1  -3  -2  5   
M   -1  -1  -2  -3  -1  0   -2  -3  -2  1   2   -1  5   
F   -2  -3  -3  -3  -2  -3  -3  -3  -1  0   0   -3  0   6   
P   -1  -2  -2  -1  -3  -1  -1  -2  -2  -3  -3  -1  -2  -4  7   
S   1   -1  1   0   -1  0   0   0   -1  -2  -2  0   -1  -2  -1  4   
T   0   -1  0   -1  -1  -1  -1  -2  -2  -1  -1  -1  -1  -2  -1  1   5   
W   -3  -3  -4  -4  -2  -2  -3  -2  -2  -3  -2  -3  -1  1   -4  -3  -2  11  
Y   -2  -2  -2  -3  -2  -1  -2  -3  2   -1  -1  -2  -1  3   -3  -2  -2  2   7   
V   0   -3  -3  -3  -1  -2  -2  -3  -3  3   1   -2  1   -1  -2  -2  0   -3  -1  4   
B   -2  -1  3   4   -3  0   

## Needleman-Wunsch

I know, I know : all these lines and still no alignment in sight. What does that title even mean ? Well, Saul B. Needleman and Christian D. Wunsch are people who came up with a clever algorithm for aligning sequences. It provides us with the alignments that get the best score, given certain conditions. It uses a scoring system like the one we've just covered, with a few twists. In order to achieve that, it creates a matrix with enough rows to fit one sequence and enough columns to fit the other : each square represents a possible alignment between two amino acids (one from each sequence). Then, it goes through every square in the matrix and decides whether a good alignment would go that way or not. In order to dertermine that it calculates

 * Scoring : the 